In [1]:
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 40.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
from sklearn.metrics import (confusion_matrix,accuracy_score,precision_score,recall_score,f1_score)
import torch.nn.functional as F

In [4]:
# element_matrix = np.load("/content/drive/MyDrive/element_matrix.npy", allow_pickle = True)
label_matrix = np.load("/content/drive/MyDrive/label_matrix.npy", allow_pickle = True)
total_elements = len(label_matrix)
m = total_elements
cos = nn.CosineSimilarity(dim=-1, eps=1e-6)
similarity_matrix = np.zeros((total_elements, total_elements))
print(label_matrix.shape)

(15339,)


In [5]:
import scipy.io

# mat_data = scipy.io.loadmat('/content/drive/MyDrive/Emotic/basic/Feature/Gabor.mat')
mat_data = scipy.io.loadmat('/content/drive/MyDrive/Emotic/basic/Feature/HOG.mat')
print(mat_data.keys())
var1 = mat_data['HOG_train']
var2 = mat_data['HOG_test']
var1 = torch.tensor(var1)
var2 = torch.tensor(var2)
feats = torch.cat((var1, var2), dim = 0)
feats.size()

dict_keys(['__header__', '__version__', '__globals__', 'HOG_train', 'HOG_test'])


torch.Size([15339, 4000])

In [6]:
# for i1 in tqdm(range(total_elements)):
#     for i2 in range(i1, total_elements):
#         input1 = element_matrix[i1]
#         input2 = element_matrix[i2]

#         output = cos(input1, input2)

#         similarity_matrix[i1, i2] = output.item()
#         similarity_matrix[i2, i1] = similarity_matrix[i1, i2]

# file_path = '/content/drive/MyDrive/similarity_matrix.npy'
# np.save(file_path, similarity_matrix)

In [7]:
similarity_matrix = np.load('/content/drive/MyDrive/similarity_matrix.npy')
similarity_matrix.shape

(15339, 15339)

In [8]:
G=nx.Graph()
for i in range(m):
  G.add_node(i)
for i in tqdm(range(m)):
  for j in range(i,m):
    if(similarity_matrix[i][j]>0.93):
      if(i!=j):
        G.add_edge(i,j,weight=similarity_matrix[i][j])
        G.add_edge(j,i,weight=similarity_matrix[i][j])
print(G.number_of_edges()/m)
print(nx.number_of_isolates(G))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

100%|██████████| 15339/15339 [03:18<00:00, 77.12it/s]  

2993.4639155094856
36


In [9]:
a=list( list(i) for i in list(G.edges()))
edge_index=torch.tensor(a)
edge_idxx=torch.transpose(edge_index,0,1)
edge_weight1=nx.get_edge_attributes(G,"weight")
edge_weight4=list(edge_weight1.values())
edge_weight2=torch.tensor(edge_weight4,dtype=torch.float32)

In [10]:
var1.size()

torch.Size([12271, 4000])

In [17]:
t1=[]
t2=[]
for i in range(m):
  if(i<=11000):
    t1.append(True)
    t2.append(False)
  else:
    t1.append(False)
    t2.append(True)
from sklearn.utils import shuffle
train_mask,test_mask=shuffle(t1,t2,random_state=3)
train_mask=torch.tensor(train_mask)
test_mask=torch.tensor(test_mask)

In [12]:
target=np.zeros((m))
for i in range(m):
  target[i] = label_matrix[i]
target=torch.tensor(target,dtype=torch.long)
target=F.one_hot(target,num_classes=7)
target=target.type(torch.FloatTensor)
target=target.long()
target=target.to(device)

In [19]:
class GCN2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=GCNConv(4000,50)
        # self.conv2=GCNConv(100,20)
        self.fc1 = nn.Linear(50,7)

    def forward(self, x, edge_index,edge_weight):
        # print(x.size(), edge_index.size(), edge_weight.size())
        x = self.conv1(x,edge_index,edge_weight)
        # print(x.size(), edge_index.size(), edge_weight.size())
        # x = self.conv2(x,edge_index,edge_weight)
        # print(x.size(), edge_index.size(), edge_weight.size())
        x = self.fc1(x).softmax(dim=1)
        return x
modelA = GCN2().to(device).float()

In [25]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelA.parameters(), lr=0.000001)
maxp=[]
maxf=[]
maxr=[]
d1=0
d2=0
d3=0

In [26]:
def accuracy(pred_y,y):
  return ((pred_y==y).sum())/len(pred_y)
embeddings = []
outputs = []
losses=[]
acc=[]
target=target.type(torch.float)
for epoch in tqdm(range(400)):
    optimizer.zero_grad()
    feats = feats.to(device).float()
    edge_idxx = edge_idxx.to(device).float()
    edge_idxx = edge_idxx.int()
    edge_weight2 = edge_weight2.to(device).float()
    z = modelA(feats,edge_idxx,edge_weight2)
    # print(z[train_mask].size(), target[train_mask].size())
    loss = criterion(z[train_mask],target[train_mask])
    acc = accuracy(z[test_mask].argmax(dim=1),target[test_mask].argmax(dim=1))
    loss.backward(retain_graph=True)
    optimizer.step()
    if epoch % 10 == 0 :
        print(f'Epoch {epoch:>3} | Loss: {loss:.2f} | Acc: {acc*100:.2f}%')
    if(epoch>80):
     a=precision_score(target[test_mask].argmax(dim=1).cpu(),z[test_mask].argmax(dim=1).cpu(), average = 'macro')
     b=f1_score(target[test_mask].argmax(dim=1).cpu(),z[test_mask].argmax(dim=1).cpu(), average = 'macro')
     c=recall_score(target[test_mask].argmax(dim=1).cpu(),z[test_mask].argmax(dim=1).cpu(), average = 'macro')
     if (a>d1):
        maxp.append([a,b,c,acc*100])
        d1=a
        embeddings1=z
     if (b>d2):
        maxf.append([a,b,c,acc*100])
        d2=b
        embeddings2=z
     if (c>d3):
        maxr.append([a,b,c,acc*100])
        d3=c
        embeddings3=z

  0%|          | 1/400 [00:00<01:08,  5.84it/s]

Epoch   0 | Loss: 1.79 | Acc: 39.37%


  3%|▎         | 11/400 [00:01<01:02,  6.27it/s]

Epoch  10 | Loss: 1.79 | Acc: 39.37%


  5%|▌         | 21/400 [00:02<01:00,  6.27it/s]

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 93%|█████████▎| 373/400 [00:55<00:04,  6.66it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 94%|█████████▎| 374/400 [00:55<00:03,  6.67it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 94%|

Epoch 380 | Loss: 1.79 | Acc: 39.37%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 96%|█████████▌| 383/400 [00:56<00:02,  6.66it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 96%|█████████▌| 384/400 [00:56<00:02,  6.68it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 96%|

Epoch 390 | Loss: 1.79 | Acc: 39.37%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 98%|█████████▊| 393/400 [00:58<00:01,  6.67it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 98%|█████████▊| 394/400 [00:58<00:00,  6.66it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 99%|